#AI - OCR

### Required package

In [ ]:
!git clone https://github.com/aziz-ullah-khan/EasyOCR.git  
#https://github.com/TopCode111/AI-OCR

### Importing required packages

In [2]:
#change directory to import train file accessories
%cd f:/Git/AI-OCR/trainer     

f:\Git\AI-OCR\trainer


In [3]:
import shutil
import os
import pandas as pd
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
from sklearn.model_selection import train_test_split

### Preparing training Data

In [9]:
# create directories for training and evaluation
%mkdir 'f:/Git/AI-OCR/trainer/all_data/ja_train_filtered/'
%mkdir 'f:/Git/AI-OCR/trainer/all_data/ja_val/'

In [3]:
BASE_DIR = 'f:/Git/AI-OCR/1moji-ground-truth/'       # Base directory of the unzip folder
target_dir_training = 'f:/Git/AI-OCR/trainer/all_data/ja_train_filtered/'
target_dir_val = 'f:/Git/AI-OCR/trainer/all_data/ja_val/'
training_images_names = []     
training_text_data = []

training_files = os.listdir(BASE_DIR)

for training_file in training_files:
  if training_file.endswith(('.jpg', '.png', 'jpeg', 'tif')):
    image_name_no_extension = training_file.split('.')[0]
    training_text_file = image_name_no_extension + '.gt.txt'
    training_text_file_directory = BASE_DIR + training_text_file
   # print(training_text_file_directory)
    if os.path.isfile(training_text_file_directory):
      with open(training_text_file_directory) as f:
        training_images_names.append(training_file)
        training_text_data.append(f.read())
    else:
      pass
      #print ("text File does not exist!") 

Training and Validation Data Preparing

In [4]:
data = pd.DataFrame()
data['filename'] = training_images_names
data['words'] = training_text_data

training_data, validation_data = train_test_split(data, test_size=0.1)

print('Total files: ', len(training_images_names))
print('Total training files: ', len(training_data))
print('Total val files: ', len(validation_data))



for file in training_data['filename']:
  shutil.copy2(os.path.join(BASE_DIR, file), target_dir_training)

for file in validation_data['filename']:
  shutil.copy2(os.path.join(BASE_DIR, file), target_dir_val)

# Save the training data to csv
training_data.to_csv(target_dir_training + 'labels.csv', encoding = 'utf-8', index = False)
validation_data.to_csv(target_dir_val + 'labels.csv', encoding = 'utf-8', index = False)

Total files:  455962
Total training files:  410365
Total val files:  45597


## Training the Model

In [4]:
cudnn.benchmark = True
cudnn.deterministic = False

In [5]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep=',', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt


In [6]:
opt = get_config("config_files/ja_filtered_config.yaml")
train(opt, amp=False)


Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['ja_train_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: ja_train_filtered
all_data/ja_train_filtered
sub-directory:	/ja_train_filtered	 num samples: 2
num total samples of ja_train_filtered: 2 x 1.0 (total_data_usage_ratio) = 2
num samples of ja_train_filtered per batch: 1 x 1.0 (batch_ratio) = 1
--------------------------------------------------------------------------------
Total_batch_size: 1 = 1
--------------------------------------------------------------------------------
dataset_root:    all_data/ja_val	 dataset: /
all_data/ja_val/
sub-directory:	/.	 num samples: 2
--------------------------------------------------

f:\Git\AI-OCR\venv\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


training time:  20.31739568710327
[10/30] Train loss: 219.66766, Valid loss: 3.84111, Elapsed_time: 20.32040
Current_accuracy : 50.000, Current_norm_ED  : 0.5000
Best_accuracy    : 50.000, Best_norm_ED     : 0.5000


UnboundLocalError: local variable 'start' referenced before assignment

## Loading the Trained Model

Copy custom_model.py custom_model.yaml  to the root directory: /root/EasyOCR/user_network/

copy custom_model.pth to /root/EasyOCR/model/


In [ ]:
!cp /root/EasyOCR/user_network/custom_model.yaml /root/.EasyOCR/user_network/custom_model.yaml
!cp /root/EasyOCR/user_network/custom_model.py /root/.EasyOCR/user_network/custom_model.py
!cp /root/EasyOCR/model/custom_model.pth /root/.EasyOCR/model/custom_model.pth

In [ ]:
import easyocr
reader = easyocr.Reader(['ja'], recog_network='custom_model')
result = reader.readtext('/content/EasyOCR/trainer/all_data/ja_val/tate001_20211203101858_00001_page_1_character_102_0.tif', detail = 0)  # remove detail =0 for details.. 

In [ ]:
result = reader.readtext('/content/name.jpg', detail = 0)  # remove detail =0 for details.. 
result

['獅', '秀', '腫', '腫', '物']

In [ ]:
!pip install tifffile 
!pip install imagecodecs
!pip install imagecodecs-lite